In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd

from junifer.storage import HDF5FeatureStorage
from sklearn.kernel_ridge import KernelRidge
from julearn import run_cross_validation
from julearn.pipeline import PipelineCreator
from julearn.utils import configure_logging
from pprint import pprint
from sklearn.model_selection import RepeatedKFold
from julearn.config import set_config
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
# 80 specific subjects same as the paper
train_subs = pd.read_csv(
    "MMP_HCP_80_subs_componentscoreestimation.txt", header=None
).values.flatten().astype(str)  # 80 subjects

# 753 specific subject for main analysis same as the paper
test_subs = pd.read_csv(
    "MMP_HCP_753_subs.txt", header=None
).values.flatten().astype(str)  # 753 subjects

columns= ['PicSeq_Unadj','CardSort_Unadj','Flanker_Unadj','PMAT24_A_CR','ReadEng_Unadj','PicVocab_Unadj','ProcSpeed_Unadj','VSPLOT_TC','SCPT_SEN','SCPT_SPEC','IWRD_TOT','ListSort_Unadj','MMSE_Score',
                     'PSQI_Score','Endurance_Unadj','Dexterity_Unadj','Strength_Unadj','Odor_Unadj','PainInterf_Tscore','Taste_Unadj','Mars_Final','Emotion_Task_Face_Acc','Language_Task_Math_Avg_Difficulty_Level',
                     'Language_Task_Story_Avg_Difficulty_Level','Relational_Task_Acc','Social_Task_Perc_Random','Social_Task_Perc_TOM','WM_Task_Acc','NEOFAC_A','NEOFAC_O','NEOFAC_C','NEOFAC_N','NEOFAC_E','ER40_CR','ER40ANG','ER40FEAR',
                     'ER40HAP','ER40NOE','ER40SAD','AngAffect_Unadj','AngHostil_Unadj','AngAggr_Unadj','FearAffect_Unadj','FearSomat_Unadj','Sadness_Unadj','LifeSatisf_Unadj','MeanPurp_Unadj','PosAffect_Unadj','Friendship_Unadj',
                     'Loneliness_Unadj','PercHostil_Unadj','PercReject_Unadj','EmotSupp_Unadj','InstruSupp_Unadj','PercStress_Unadj','SelfEff_Unadj','DDisc_AUC_40K','GaitSpeed_Comp']
# Load the dataset
full_df = pd.read_csv("Behavioral_Data", index_col="Subject")[columns]
full_df.index = full_df.index.astype(str)
test_df = full_df.loc[test_subs]
train_df = full_df.loc[train_subs]

scaler = StandardScaler()
imputer = IterativeImputer(max_iter=20, random_state=0)
train_df_imputed = imputer.fit_transform(train_df)
train_df_scaled = scaler.fit_transform(train_df_imputed)
test_df_imputed = imputer.transform(test_df)
test_df_scaled = scaler.transform(test_df_imputed)


In [29]:
# Define Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Linear(16,8),
            nn.ReLU(),
            nn.Linear(8, encoding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 8),
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(32, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Model parameters
input_dim = full_df.shape[1]
encoding_dim = 3  #Reduced dimensionality
model = Autoencoder(input_dim, encoding_dim)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01,weight_decay=1e-4)

# Training
epochs = 500
batch_size = 16
X_train_tensor = torch.tensor(train_df_scaled, dtype=torch.float32)

def evaluate(model, test, criterion):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        X_test_tensor = torch.tensor(test, dtype=torch.float32)
        outputs = model(X_test_tensor)  # Get decoded output
        val_loss = criterion(outputs[1], X_test_tensor)  # Compare reconstructed data
    return val_loss.item()


for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    reconstructed = model(X_train_tensor)[1]
    loss = criterion(reconstructed, X_train_tensor)
    loss.backward()
    optimizer.step()

    # Validation loss
    val_loss = evaluate(model, test_df_scaled, criterion) 

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}")



Epoch 1/500, Loss: 1.0121, Validation Loss: 0.9083
Epoch 2/500, Loss: 1.0085, Validation Loss: 0.9039
Epoch 3/500, Loss: 1.0053, Validation Loss: 0.9004
Epoch 4/500, Loss: 1.0022, Validation Loss: 0.8978
Epoch 5/500, Loss: 1.0015, Validation Loss: 0.8951
Epoch 6/500, Loss: 0.9983, Validation Loss: 0.8919
Epoch 7/500, Loss: 0.9958, Validation Loss: 0.8875
Epoch 8/500, Loss: 0.9918, Validation Loss: 0.8804
Epoch 9/500, Loss: 0.9873, Validation Loss: 0.8692
Epoch 10/500, Loss: 0.9784, Validation Loss: 0.8526
Epoch 11/500, Loss: 0.9656, Validation Loss: 0.8311
Epoch 12/500, Loss: 0.9463, Validation Loss: 0.8125
Epoch 13/500, Loss: 0.9363, Validation Loss: 0.8086
Epoch 14/500, Loss: 0.9247, Validation Loss: 0.7978
Epoch 15/500, Loss: 0.9189, Validation Loss: 0.7901
Epoch 16/500, Loss: 0.8974, Validation Loss: 0.7926
Epoch 17/500, Loss: 0.8833, Validation Loss: 0.7970
Epoch 18/500, Loss: 0.8811, Validation Loss: 0.8003
Epoch 19/500, Loss: 0.8678, Validation Loss: 0.8048
Epoch 20/500, Loss: 0

In [4]:
# Encode the behavioral data to lower dimensions
with torch.no_grad():
    X_train_reduced = model.encoder(X_train_tensor).numpy()
    X_test_tensor = torch.tensor(test_df_scaled, dtype=torch.float32)
    X_test_reduced = model.encoder(X_test_tensor).numpy()

test_df_reduced = pd.DataFrame(
        X_test_reduced,
        index=test_df.index,
        columns=["component_1", "component_2", "component_3"],
    )


In [5]:
# %% Feature generation
storage = HDF5FeatureStorage("features.hdf5")
df_features = storage.read_df('BOLD_Schaefer400x17_functional_connectivity')

# Group by 'subject' and calculate the mean across REST1/LR, REST1/RL, REST2/LR, REST2/RL 
df_features = df_features.groupby('subject').mean()

# merge the dataframe
df_features.index.name= "Subject"
df_data = df_features.join(test_df_reduced, how="inner")

# %%
# Regression model training
# Step 1: Seperate features (X) and targets (y)
X = [x for x in df_data.columns if "~" in x]
X = [x for x in X if x.split("~")[0] != x.split("~")[1]]
y = "component_2"
X_types = {
    "features":[".*~.*"],
} 
# %%
# Create the pipeline that will be used to predict the target.
kernel_ridge_model = KernelRidge()
creator = PipelineCreator(problem_type="regression",apply_to="features")
creator.add("zscore")
creator.add(
    kernel_ridge_model, 
    alpha=10, 
    kernel='linear', 
    )
# %%
# Evaluate the model within the cross validation.
rkf = RepeatedKFold(n_splits=10,n_repeats=5,random_state=42)
scores_tuned, model_tuned = run_cross_validation(
    X=X,
    y=y,
    X_types=X_types,
    data=df_data,
    model=creator, 
    return_estimator="all",
    cv=rkf,
    scoring= ['r2_corr','r_corr'],
    n_jobs = -1,
)
print("r2_corr:\n",scores_tuned["test_r2_corr"])
print("r_corr:\n",scores_tuned["test_r_corr"])
print(f"Scores with best hyperparameter: {scores_tuned['test_r_corr'].mean()}")


2025-01-16 15:18:10,845 [ WARNING] /home/haotsung/HCP_behavioral_prediction/julearn_hcp/lib/python3.11/site-packages/julearn/prepare.py:195: RuntimeWarning: The dataframe has 80203 columns. Checking X for consistency might take a while. To skip this checks, set the config flag `disable_x_check` to `True`.
  warn_with_log(

2025-01-16 15:37:54,991 [ WARNING] /home/haotsung/HCP_behavioral_prediction/julearn_hcp/lib/python3.11/site-packages/julearn/prepare.py:471: RuntimeWarning: X has 79800 columns. Checking X_types for consistency might take a while. To skip this checks, set the config flag `disable_xtypes_check` to `True`.
  warn_with_log(



r2_corr:
 0     8.941516e-04
1     3.442660e-03
2     1.162774e-02
3     3.633204e-04
4     3.177245e-03
5     1.165064e-02
6     1.066333e-04
7     1.946646e-03
8     7.974767e-03
9     8.957556e-03
10    2.904341e-03
11    3.149623e-03
12    2.982044e-05
13    1.769274e-03
14    1.012621e-02
15    4.472219e-09
16    5.508813e-04
17    1.977997e-02
18    1.910536e-02
19    2.318740e-02
20    2.005093e-02
21    1.342179e-04
22    1.102300e-02
23    6.934613e-03
24    2.174031e-02
25    1.848244e-02
26    4.775650e-03
27    2.921824e-04
28    4.770307e-03
29    5.983361e-04
30    9.828886e-03
31    6.030415e-02
32    8.826164e-04
33    1.536766e-03
34    1.223135e-02
35    9.692623e-04
36    6.620950e-03
37    8.303500e-05
38    2.040788e-03
39    5.181090e-04
40    8.778065e-03
41    1.272808e-02
42    8.119800e-03
43    4.959147e-04
44    8.442090e-03
45    7.363955e-03
46    2.469838e-04
47    1.240929e-03
48    1.623200e-02
49    1.305725e-04
Name: test_r2_corr, dtype: float64
r_cor